In [514]:
import pandas as pd
import json

df_csv = pd.read_csv('UNSPSC_English.csv')
df_xlsx = pd.read_excel('Avalara_goods_and_services.xlsx')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3194: DtypeWarning: Columns (1,2,5,8,9,12,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [515]:
df_xlsx = df_xlsx.rename(columns={'Unnamed: 0':'AvaTax System Tax Code', 'AvaTax System Tax Codes':'AvaTax System Tax Code Description',
                                 'Unnamed: 2':'Additional AvaTax System Tax Code Information'})
df_xlsx = df_xlsx[2:]
df_xlsx

,AvaTax System Tax Code,AvaTax System Tax Code Description,Additional AvaTax System Tax Code Information
2,D0000000,Digital goods,Digital goods are generally viewed as download...
3,D9999999,Temporary unmapped digital goods sku - taxable...,A digital good that has not been mapped to a s...
4,DA010000,Digital goods: advertising material (digital o...,"Tangible personal property (i.e. designs, artw..."
5,DA030000,Digital goods: art (digital only -- no physica...,Canned artwork that is transferred to the cust...
6,DA040000,Digital goods: audio (digital only -- no physi...,Audio (i.e. music or spoken word) that is tran...
...,...,...,...
2541,SY010200,Yoga classes - in facilities that offer servic...,NaN
2542,SL090200,Lodging accommodations,NaN
2543,ST110000,Travel agency,NaN
2544,ST112506,Travel agency fees,NaN


In [516]:
# import nltk and download resources
import nltk
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/mikec/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mikec/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [517]:
# parse the stop words and special characters out
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
stop_words.add(':')
stop_words.add('--')
stop_words.add('-')
stop_words.add('(')
stop_words.add(')')
stop_words.add('/')
stop_words.add('*')
stop_words.add(';')
stop_words.add('.')
stop_words.add('i.e')
stop_words.add('A')
stop_words.add('a')
stop_words.add(',')

In [518]:
def createParsedColumns(file, col1, col2, start_index, new_col):
    for index1, description1 in enumerate(file[col1], start_index):
        if not pd.isna(description1):
            word_tokens1 = word_tokenize(description1)

            filtered_sentence1 = []
            for w1 in word_tokens1:
                if w1 not in stop_words:
                    filtered_sentence1.append(w1.lower())

            # remove empty strings
            while "" in filtered_sentence1:
                filtered_sentence1.remove("")

            # updating record in col1
            file.at[index1, col1]= ','.join(filtered_sentence1)
        else:
            file.at[index1, col1]= ''

    for index2, description2 in enumerate(file[col2], start_index):
        if not pd.isna(description2):
            word_tokens2 = word_tokenize(description2)

            filtered_sentence2 = []
            for w2 in word_tokens2:
                if w2 not in stop_words:
                    filtered_sentence2.append(w2.lower())

            # remove empty strings
            while "" in filtered_sentence2:
                filtered_sentence2.remove("")

            # updating record in col2
            file.at[index2, col2]= ','.join(filtered_sentence2)
        else:
            file.at[index2, col2]= ''

    # create 3rd column
    new_column_series = []
    for index3, description3 in enumerate(file[col2], start_index):
        temp1 = file[col1].loc[index3].split(',')

        if not pd.isna(description3):
            temp2 = file[col2].loc[index3].split(',')
            temp3 = temp1 + temp2
            temp3 = set(temp3)

            if '' in temp3:
                temp3.remove('')

            new_column_series.append(list(temp3))
        else:
            new_column_series.append(list(temp1))

    # insert new column
    file.loc[:,new_col] = new_column_series

In [519]:
column_name = 'AvaTax System Tax Code Description'
column_name_2 = 'Additional AvaTax System Tax Code Information'
new_col_name = 'tax_parsed'
createParsedColumns(df_xlsx, column_name, column_name_2, 2, new_col_name)
df_xlsx

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,AvaTax System Tax Code,AvaTax System Tax Code Description,Additional AvaTax System Tax Code Information,tax_parsed
2,D0000000,"digital,goods","digital,goods,generally,viewed,downloadable,it...","[downloadable, goods, photographs, sold, zip, ..."
3,D9999999,"temporary,unmapped,digital,goods,sku,taxable,d...","digital,good,mapped,specific,digital,good,tax,...","[specific, impose, goods, system, tax, exempti..."
4,DA010000,"digital,goods,advertising,material,digital,phy...","tangible,personal,property,designs,artwork,pho...","[associated, means, personal, goods, customer,..."
5,DA030000,"digital,goods,art,digital,physical,media","canned,artwork,transferred,customer,electronic...","[canned, means, artwork, goods, customer, phys..."
6,DA040000,"digital,goods,audio,digital,physical,media","audio,music,spoken,word,transferred,customer,e...","[audio, means, word, goods, customer, physical..."
...,...,...,...,...
2541,SY010200,"yoga,classes,facilities,offer,services,yoga,cl...",,"[services, facilities, classes, yoga, offer]"
2542,SL090200,"lodging,accommodations",,"[accommodations, lodging]"
2543,ST110000,"travel,agency",,"[agency, travel]"
2544,ST112506,"travel,agency,fees",,"[agency, travel, fees]"


In [520]:
tc_hash_map = {}

new_col_name = 'tax_parsed'

for index, tax_code in enumerate(df_xlsx['AvaTax System Tax Code'], 2):

    # if tax_code = NaN
    if isinstance(tax_code, float):
        prefix = df_xlsx["AvaTax System Tax Code"].loc[index + 1]
        level_1 = prefix[0:2]
        level_2 = prefix[2:4]
        level_3 = "0000"
        df_xlsx["AvaTax System Tax Code"].loc[index] = prefix[0:4]+"0000"
    else:
        level_1 = tax_code[0:2]
        level_2 = tax_code[2:4]
        level_3 = tax_code[4:8]
    

    if level_1 not in tc_hash_map:
        temp_dict0 = {level_3: df_xlsx[new_col_name].loc[index]}
        temp_dict = {level_2: [df_xlsx[new_col_name].loc[index], temp_dict0]}
        tc_hash_map[level_1] = [df_xlsx[new_col_name].loc[index], temp_dict]
    elif level_2 not in tc_hash_map[level_1][1]:
        temp_dict0 = {level_3: df_xlsx[new_col_name].loc[index]}
        tc_hash_map[level_1][1][level_2] = [df_xlsx[new_col_name].loc[index], temp_dict0]
    elif level_3 not in tc_hash_map[level_1][1][level_2][1]:
        tc_hash_map[level_1][1][level_2][1][level_3] = df_xlsx[new_col_name].loc[index]

print(json.dumps(tc_hash_map, sort_keys=True, indent=2))

{
  "D0": [
    [
      "downloadable",
      "goods",
      "photographs",
      "sold",
      "zip",
      "websites",
      "software",
      "artwork",
      "otherwise",
      "movies",
      "examples",
      "generally",
      "include",
      "pdfs",
      "transferred",
      "e-books",
      "files",
      "electronically",
      "items",
      "would",
      "computer",
      "viewed",
      "music",
      "digital"
    ],
    {
      "00": [
        [
          "downloadable",
          "goods",
          "photographs",
          "sold",
          "zip",
          "websites",
          "software",
          "artwork",
          "otherwise",
          "movies",
          "examples",
          "generally",
          "include",
          "pdfs",
          "transferred",
          "e-books",
          "files",
          "electronically",
          "items",
          "would",
          "computer",
          "viewed",
          "music",
          "digital"
        ],
        {
  

In [521]:
tc_table_lvl_1 = {}
tc_table_lvl_2 = {}
tc_table_lvl_3 = {}

new_col_name = 'tax_parsed'

for index, tax_code in enumerate(df_xlsx['AvaTax System Tax Code'], 2):
    # if tax_code = NaN
    if isinstance(tax_code, float):
        prefix = df_xlsx["AvaTax System Tax Code"].loc[index + 1]
        level_1 = prefix[0:2]
        level_2 = prefix[2:4]
        level_3 = "0000"
        df_xlsx["AvaTax System Tax Code"].loc[index] = prefix[0:4]+"0000"
    else:
        level_1 = tax_code[0:2]
        level_2 = tax_code[2:4]
        level_3 = tax_code[4:8]


    if level_1 not in tc_table_lvl_1:
        tc_table_lvl_1[level_1] = df_xlsx[new_col_name].loc[index]

    if level_1 + level_2 not in tc_table_lvl_2:
        tc_table_lvl_2[level_1+level_2] = df_xlsx[new_col_name].loc[index]

    if level_1 + level_2 + level_3 not in tc_table_lvl_3:
        tc_table_lvl_3[level_1+level_2+level_3] = df_xlsx[new_col_name].loc[index]

print(json.dumps(tc_table_lvl_1, sort_keys=True, indent=2))
print(json.dumps(tc_table_lvl_2, sort_keys=True, indent=2))
print(json.dumps(tc_table_lvl_3, sort_keys=True, indent=2))

{
  "D0": [
    "downloadable",
    "goods",
    "photographs",
    "sold",
    "zip",
    "websites",
    "software",
    "artwork",
    "otherwise",
    "movies",
    "examples",
    "generally",
    "include",
    "pdfs",
    "transferred",
    "e-books",
    "files",
    "electronically",
    "items",
    "would",
    "computer",
    "viewed",
    "music",
    "digital"
  ],
  "D9": [
    "specific",
    "impose",
    "goods",
    "system",
    "tax",
    "exemption",
    "entity",
    "taxable",
    "reviewed",
    "good",
    "accurately",
    "unmapped",
    "applicable",
    "using",
    "cases",
    "adequately",
    "use",
    "purpose",
    "unless",
    "getting",
    "temporary",
    "default",
    "assigned",
    "sku",
    "limited",
    "mapped",
    "code",
    "base",
    "digital"
  ],
  "DA": [
    "associated",
    "means",
    "personal",
    "goods",
    "customer",
    "photos",
    "tangible",
    "services",
    "etc",
    "artwork",
    "physical",
    "media

In [522]:
# Convert row to header and remove the row
df_csv = df_csv.rename(columns=df_csv.iloc[8])
df_csv = df_csv.loc[9:]
df_csv = df_csv.drop(columns = ['Version', 'Key', 'Column1', 'Synonym', 'Acronym'])
df_csv

,Segment,Segment Title,Segment Definition,Family,Family Title,Family Definition,Class,Class Title,Class Definition,Commodity,Commodity Title,Definition
9,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10101500,Livestock,NaN,NaN,NaN,NaN
12,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10101500,Livestock,NaN,10101501,Cats,NaN
13,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10101500,Livestock,NaN,10101502,Dogs,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
156402,95000000,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,95140000.0,Prefabricated buildings and structures,Buildings and structures used for various farm...,95141900.0,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,NaN,NaN,NaN
156403,95000000,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,95140000.0,Prefabricated buildings and structures,Buildings and structures used for various farm...,95141900.0,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,95141901.0,Medical unit,A building or structure where medical equipmen...
156404,95000000,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,95140000.0,Prefabricated buildings and structures,Buildings and structures used for various farm...,95141900.0,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,95141902.0,Laboratory unit,A building or structure where laboratory equip...
156405,95000000,Land and Buildings and Structures and Thorough...,Land and waterways and related real property p...,95140000.0,Prefabricated buildings and structures,Buildings and structures used for various farm...,95141900.0,Prefabricated medical buildings and structures,Buildings and structures used for medical purp...,95141903.0,Dental unit,A building or structure where dental equipment...


In [523]:
column_name = 'Segment Title'
column_name_2 = 'Segment Definition'
column_name_3 = 'Family Title'
column_name_4 = 'Family Definition'
column_name_5 = 'Class Title'
column_name_6 = 'Class Definition'
column_name_7 = 'Commodity Title'
column_name_8 = 'Definition'
createParsedColumns(df_csv, column_name, column_name_2, 9, 'segment_parsed')
createParsedColumns(df_csv, column_name_3, column_name_4, 9, 'family_parsed')
createParsedColumns(df_csv, column_name_5, column_name_6, 9, 'class_parsed')
createParsedColumns(df_csv, column_name_7, column_name_8, 9, 'commodity_parsed')
df_csv

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,Segment,Segment Title,Segment Definition,Family,Family Title,Family Definition,Class,Class Title,Class Definition,Commodity,Commodity Title,Definition,segment_parsed,family_parsed,class_parsed,commodity_parsed
9,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",NaN,,,NaN,,,NaN,,,"[used, this, domesticated, plants, includes, s...",[],[],[]
10,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",10100000,"live,animals",,NaN,,,NaN,,,"[used, this, domesticated, plants, includes, s...","[live, animals]",[],[]
11,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",10100000,"live,animals",,10101500,livestock,,NaN,,,"[used, this, domesticated, plants, includes, s...","[live, animals]",[livestock],[]
12,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",10100000,"live,animals",,10101500,livestock,,10101501,cats,,"[used, this, domesticated, plants, includes, s...","[live, animals]",[livestock],[cats]
13,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",10100000,"live,animals",,10101500,livestock,,10101502,dogs,,"[used, this, domesticated, plants, includes, s...","[live, animals]",[livestock],[dogs]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156402,95000000,"land,buildings,structures,thoroughfares","land,waterways,related,real,property,physical,...",95140000.0,"prefabricated,buildings,structures","buildings,structures,used,various,farm,related...",95141900.0,"prefabricated,medical,buildings,structures","buildings,structures,used,medical,purposes,pre...",NaN,,,"[real, thoroughfares, assets, buildings, relat...","[used, various, location, purposes, pre-constr...","[used, location, purposes, pre-constructed, bu...",[]
156403,95000000,"land,buildings,structures,thoroughfares","land,waterways,related,real,property,physical,...",95140000.0,"prefabricated,buildings,structures","buildings,structures,used,various,farm,related...",95141900.0,"prefabricated,medical,buildings,structures","buildings,structures,used,medical,purposes,pre...",95141901.0,"medical,unit","building,structure,medical,equipment,house,use...","[real, thoroughfares, assets, buildings, relat...","[used, various, location, purposes, pre-constr...","[used, location, purposes, pre-constructed, bu...","[used, building, house, tests, experiments, un..."
156404,95000000,"land,buildings,structures,thoroughfares","land,waterways,related,real,property,physical,...",95140000.0,"prefabricated,buildings,structures","buildings,structures,used,various,farm,related...",95141900.0,"prefabricated,medical,buildings,structures","buildings,structures,used,medical,purposes,pre...",95141902.0,"laboratory,unit","building,structure,laboratory,equipment,house,...","[real, thoroughfares, assets, buildings, relat...","[used, various, location, purposes, pre-constr...","[used, location, purposes, pre-constructed, bu...","[used, building, house, tests, experiments, la..."
156405,95000000,"land,buildings,structures,thoroughfares","land,waterways,related,real,property,physical,...",95140000.0,"prefabricated,buildings,structures","buildings,structures,used,various,farm,related...",95141900.0,"prefabricated,medical,buildings,structures","buildings,structures,used,medical,purposes,pre...",95141903.0,"dental,unit","building,structure,dental,equipment,house,used...","[real, thoroughfares, assets, buildings, relat...","[used, various, location, purposes, pre-constr...","[used, location, purposes, pre-constructed, bu...","[used, building, dental, house, tests, experim..."
